# Recherche Vectorielle avec Milvus

Ce notebook démontre comment effectuer une recherche sémantique dans la base vectorielle Milvus.

In [ ]:
from pymilvus import connections, Collection
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np

In [ ]:
# Connexion à Milvus
connections.connect("default", host="127.0.0.1", port="19530")
collection = Collection("music_embeddings")
collection.load()
print(f"Collection loaded. Num entities: {collection.num_entities}")

In [ ]:
# Chargement du modèle
model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
def search_by_text(query: str, top_k: int = 5):
    # 1. Vectoriser la requête
    query_embedding = model.encode([query])
    
    # 2. Recherche Milvus
    search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
    results = collection.search(
        data=query_embedding,
        anns_field="embedding",
        param=search_params,
        limit=top_k,
        output_fields=["title", "artist"]
    )
    
    # 3. Formater les résultats
    output = []
    for hits in results:
        for hit in hits:
            output.append({
                "id": hit.id,
                "title": hit.entity.get("title"),
                "artist": hit.entity.get("artist"),
                "distance": hit.distance
            })
    return output

In [ ]:
# Test de recherche
query = "dark ambient"
results = search_by_text(query)
pd.DataFrame(results)